In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
plt.style.use('bmh')

# Weibull AFT

In [3]:
df = pd.read_csv('./data/veteran.csv')

In [4]:
df.head()

,Unnamed: 0,trt,celltype,time,status,karno,diagtime,age,prior
0,1,1,squamous,72,1,60,7,69,0
1,2,1,squamous,411,1,70,5,64,10
2,3,1,squamous,228,1,60,3,38,0
3,4,1,squamous,126,1,60,9,63,10
4,5,1,squamous,118,1,70,11,65,10


`celltype`: categorical covariate -> must converted to the num.vals using one-hot-encoding technique

In [8]:
df1 = df[df['trt']==1].drop(['trt', 'Unnamed: 0'], axis=1)
df1 = pd.get_dummies(df1, columns=['celltype'])

In [9]:
df1.head()

,time,status,karno,diagtime,age,prior,celltype_adeno,celltype_large,celltype_smallcell,celltype_squamous
0,72,1,60,7,69,0,False,False,False,True
1,411,1,70,5,64,10,False,False,False,True
2,228,1,60,3,38,0,False,False,False,True
3,126,1,60,9,63,10,False,False,False,True
4,118,1,70,11,65,10,False,False,False,True


In [11]:
from lifelines import WeibullAFTFitter
waft = WeibullAFTFitter(fit_intercept=False).fit(df1, duration_col='time', event_col='status')

In [13]:
waft.print_summary()

<lifelines.WeibullAFTFitter: fitted with 69 total observations, 5 right-censored observations>
             duration col = 'time'
                event col = 'status'
   number of observations = 69
number of events observed = 64
           log-likelihood = -362.85
         time fit was run = 2024-06-29 02:51:59 UTC

---
                             coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param   covariate                                                                                                                  
lambda_ age                  0.00       1.00       0.01            -0.02             0.03                 0.98                 1.03
        celltype_adeno       2.41      11.16       0.99             0.47             4.36                 1.60                77.88
        celltype_large       3.64      38.01       0.96             1.76             5.52                 5.81               248.81
        celltype_smallcell   3.17      23.70       0.94             1.32             5.01                 3.76               149.29
        celltype_squamous    3.57      35.51       0.94             1.73             5.41                 5.65               223.15
        diagtime            -0.00       1.00       0.02            -0.04             0.04                 0.96                 1.04
        karno                0.02       1.03       0.01             0.01             0.04                 1.01                 1.04
        prior               -0.04       0.96       0.03            -0.10             0.02                 0.91                 1.02
rho_    Intercept            0.08       1.08       0.09            -0.11             0.27                 0.90                 1.30

                             cmp to     z      p   -log2(p)
param   covariate                                          
lambda_ age                    0.00  0.19   0.85       0.24
        celltype_adeno         0.00  2.43   0.01       6.06
        celltype_large         0.00  3.80 <0.005      12.73
        celltype_smallcell     0.00  3.37 <0.005      10.38
        celltype_squamous      0.00  3.81 <0.005      12.79
        diagtime               0.00 -0.03   0.98       0.03
        karno                  0.00  3.05 <0.005       8.77
        prior                  0.00 -1.40   0.16       2.64
rho_    Intercept              0.00  0.85   0.40       1.33
---
Concordance = 0.69
AIC = 743.69
log-likelihood ratio test = 19.43 on 7 df
-log2(p) of ll-ratio test = 7.17